# **차원 축소(Dimensionality Reduction)**



---



## **1. 주성분 분석(PCA : Principal Component Analysis)**

- 데이터의 분산을 최대한 유지하면서 특성이 많은 데이터세트의 차원을 줄이는 방법
- 원본 데이터의 분산을 최대한 보존하는 새로운 축을 찾고(주성분), 그 축에 데이터를 투영(Projection)해서 차원을 축소함
- 이를 통해 시각화와 계산이 용이하고 분석이 쉽도록 하는 **선형 변환으로 차원 축소**


### **예제 : 주성분 분석 기본**

**1. 샘플 데이터 만들기**
- **make_blobs**: '덩어리(blob)' 형태의 가상 데이터를 만들어주는 함수
- 주로 클러스터링(Clustering, 군집화) 알고리즘이나 분류(Classification) 알고리즘을 테스트하고 시각적으로 이해하는 데 사용되는, 일종의 '연습용 데이터 세트 제조기'

In [ ]:
# 라이브러리 불러오기
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt

# 실습용 데이터셋 생성하기
x, y = make_blobs(n_features=10,
                  n_samples=1000,
                  centers=5,
                  random_state=2023,
                  cluster_std=1)
print(x.shape)
print(y.shape)

plt.scatter(x[:, 0], x[:, 1], c=y)
plt.show()

**2. 훈련데이터 표준화스케일링**

In [ ]:
# 라이브러리 불러오기
from sklearn.preprocessing import StandardScaler

# 데이터 표준화하기
scaler = StandardScaler()
scaler.fit(x)
std_data = scaler.transform(x)

print(std_data)

**3. 주성분 분석 수행하기**

In [ ]:
# 라이브러리 불러오기
import pandas as pd
from sklearn.decomposition import PCA

# PCA 객체로 주성분 10개 추출하기
pca = PCA(n_components=10)
reduced_data = pca.fit_transform(std_data)

# 주성분 데이터 확인하기
pca_df = pd.DataFrame(reduced_data)
pca_df.head()

In [ ]:
# 고윳값: 설명된 분산(explained variance) 값 확인하기
print(pca.explained_variance_)

print('총 분산비율: ', np.sum(pca.explained_variance_ratio_))

In [ ]:
# 기여율: 설명된 분산비율(explained variance ratio) 확인하기
print(pca.explained_variance_ratio_)

In [ ]:
# 라이브러리 불러오기
import numpy as np

# 주성분의 설명력과 기여율 구하기
index = np.array([f'pca{n+1}' for n in range(reduced_data.shape[1])])
result = pd.DataFrame({'고유값' : pca.explained_variance_,
                       '기여율' : pca.explained_variance_ratio_},
                     index=index)
result['누적기여율']  = result['기여율'].cumsum()

# 주성분의 설명력과 기여율 확인하기
display(result)

In [ ]:
# PCA 객체로 주성분 4개 추출하기
pca = PCA(n_components=4)
X_reduced = pca.fit_transform(std_data)
print(pca.explained_variance_ratio_)

In [ ]:
# 지정한 비율에 도달할 때까지 주성분을 탐색하기
pca = PCA(n_components=0.9)
reduced_data = pca.fit_transform(std_data)
print(pca.explained_variance_ratio_)

### **[실습] 과일 사진 데이터으로 주성분 분석 실습하기**
0. 과일 사진 데이터셋을 사용하여
1. 주성분 분석(PCA)으로 피처 차원 축소하고
2. 축소된 데이터로 원본 데이터 재구성하기
3. 설명된 분산 확인하기

#### **1. 주성분 분석(PCA)으로 피처 차원 축소하기**


In [ ]:
# 데이터 다운로드하기
!wget https://bit.ly/fruits_300_data -O fruits_300.npy

In [ ]:
# 필요한 라이브러리 불러오기
import numpy as np
import matplotlib.pyplot as plt

# 1.fruits_300.npy 파일 로드하기
# (300, 100, 100) : 100x100 크기의 이미지 300장.
fruits = np.load('fruits_300.npy')

# 2. 머신러닝 모델에 사용하기 위해 3차원 배열을 2차원 배열로 변경
# (300, 100, 100) -> (300, 10000): 각 이미지를 10000개의 픽셀 값을 가진 하나의 샘플로 만듦
fruits_2d = fruits.reshape(-1, 100*100)


In [ ]:
# PCA 클래스 객체 만들기
from sklearn.decomposition import PCA

pca = PCA(n_components=50) # 주성분 개수 50개
pca.fit(fruits_2d)

In [ ]:
# PCA 클래스가 찾은 주성분
print(pca.components_.shape)

In [ ]:
# 주성분을 100x100 이미지  크기로 출력하기
import matplotlib.pyplot as plt

def draw_fruits(arr, ratio=1):
    n = len(arr)    # n은 샘플 개수입니다
    # 한 줄에 10개씩 이미지를 그립니다. 샘플 개수를 10으로 나누어 전체 행 개수를 계산합니다.
    rows = int(np.ceil(n/10))
    # 행이 1개 이면 열 개수는 샘플 개수입니다. 그렇지 않으면 10개입니다.
    cols = n if rows < 2 else 10
    fig, axs = plt.subplots(rows, cols,
                            figsize=(cols*ratio, rows*ratio), squeeze=False)
    for i in range(rows):
        for j in range(cols):
            if i*10 + j < n:    # n 개까지만 그립니다.
                axs[i, j].imshow(arr[i*10 + j], cmap='gray_r')
            axs[i, j].axis('off')
    plt.show()


draw_fruits(pca.components_.reshape(-1, 100, 100))
# 이 주성분은 원본 데이터에서 가장 분산이 큰 방향을 순서대로 나타냄

In [ ]:
# 원본 데이터의 모양
print(fruits_2d.shape)

In [ ]:
# PCA transform() 메서드를 사용해 원본 데이터의 차원을 50으로 줄여본다.
fruits_pca = pca.transform(fruits_2d) # 50개의 특성을 가진 데이터로 변환됨
print(fruits_pca.shape)

#### **2. 원본 데이터 재구성하기**

In [ ]:
# 50개 차원으로 축소한 fruits_pca 데이터를 전달해 10,000(100x100)개의 특성 복원하기
fruits_inverse = pca.inverse_transform(fruits_pca)
print(fruits_inverse.shape)

In [ ]:
# 복원된 데이터 시각화하기(100x100 이미지 크기로 출력하기)
fruits_reconstruct = fruits_inverse.reshape(-1, 100, 100)

for start in [0, 100, 200]:
    draw_fruits(fruits_reconstruct[start:start+100])
    print("\n")

# 결과 이미지가 일부 흐리거나 번져보이지만
# 50개 특성에서 10,000개로 늘린 것을 감안한다면
# 50개 특성이 분산을 가장 잘 보존하도록 변화된 것을 확인할 수 있다.

In [ ]:
# 원본 데이터 이미지
# for start in [0, 100, 200]:
#     draw_fruits(fruits[start:start+100])
#     print("\n")

#### **3.설명된 분산 확인하기**

In [ ]:
# 설명된 분산(explained variance) 값 확인하기 : 고윳값
print(pca.explained_variance_)

print('총 분산비율: ', np.sum(pca.explained_variance_ratio_))

In [ ]:
# 설명된 분산 그래프로 시각화
plt.plot(pca.explained_variance_ratio_)

### **[실습] 다른 알고리즘과 함께 사용하기**
과일 사진 원본 데이터와 PCA로 축소된 데이터를 지도학습(과일 사진 분류)에 적용해 보고 어떤 차이가 있는지 확인하기
- 과일 사진 분류
- 알고리즘 : LogistricRegression

In [ ]:
# 분류 모델 지정하기
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

In [ ]:
# 타겟 지정, 0:사과, 1:파인애플, 2:바나나
target = np.array([0] * 100 + [1] * 100 + [2] * 100)

In [ ]:
# 로지스틱 회귀 모델에서 성능 가능하기 위해 교차검증 수행
from sklearn.model_selection import cross_validate

scores = cross_validate(lr, fruits_2d, target)
print(np.mean(scores['test_score']))
print(np.mean(scores['fit_time']))  # 교차 검증 폴드의 훈련시간

In [ ]:
# 로지스틱 회귀 모델에서 PCA적용한 fruits_pca 데이터로 성능 교차검증 수행
scores = cross_validate(lr, fruits_pca, target)
print(np.mean(scores['test_score']))
print(np.mean(scores['fit_time']))  # 교차 검증 폴드의 훈련시간

In [ ]:
# n_components에 주성분 갯수(정수) 대신 0~1사이의 숫자로
# PCA 클래스가 지정된 비율에 도달할때까지 자동으로 주성분을 찾도록 설정한다.
pca = PCA(n_components=0.5)  # 설명된 분산의 50%에 달하는 주성분 찾도록 설정
pca.fit(fruits_2d)

In [ ]:
# 찾은 주성분 확인하기
print(pca.n_components_)

# 결과값: 2 --> 2개의 특성만으로 원본 데이터에 있는 분산의 50%를 표현할 수 있다.

In [ ]:
# 원본 데이터 변환하기
fruits_pca = pca.transform(fruits_2d)
print(fruits_pca.shape)

In [ ]:
# 2개의 특성만 사용하고도 교차 검증의 결과가 좋은지 확인하기
scores = cross_validate(lr, fruits_pca, target)
print(np.mean(scores['test_score']))
print(np.mean(scores['fit_time']))

# 참고 : 결과 warning메시지(반복 횟수 증가) 나오지만 무시해도 좋다.

In [ ]:
# 축소된 차원 데이터(fruits_pca) 클러스터링 찾아보기
from sklearn.cluster import KMeans

km = KMeans(n_clusters=3, random_state=42)
km.fit(fruits_pca)

In [ ]:
# 클러스터별 샘플 데이터 갯수 확인
print(np.unique(km.labels_, return_counts=True))

In [ ]:
# 클러스터별 데이터 이미지로 시각화하기
for label in range(0, 3):
    draw_fruits(fruits[km.labels_ == label])
    print("\n")

In [ ]:
# 산점도로 확인하기
for label in range(0, 3):
    data = fruits_pca[km.labels_ == label]
    plt.scatter(data[:,0], data[:,1])
plt.legend(['pineapple', 'banana', 'apple'])
plt.show()

### **[실습] 과일 사진 데이터셋을 3차원 Tensorflow 임베딩 공간에서 확인하기**

과일 사진 데이터셋을 고차원의 벡터 공간(임베딩 공간)에서 데이터들이 어떻게 분포하고 서로 어떤 관계를 맺고 있는지 시각적으로 탐색하시오.

- Tensorflow Embeddng Project
    - https://projector.tensorflow.org/
    - TensorFlow Projector는 단어, 이미지, 또는 그 외의 다양한 데이터가 고차원의 벡터 공간(임베딩 공간)에서 어떻게 분포하고 서로 어떤 관계를 맺고 있는지 시각적으로 탐색하고 분석할 수 있도록 Google에서 만든 강력한 웹 기반 도구


**1. embedding project용 데이터로 변환하기**
- 코랩에서 생성된 파일--> PC로 다운로드하기
    - vector파일 : fruits_vectors.tsv
    - metadata파일 : fruits_metadata.tsv

In [ ]:
import numpy as np
import pandas as pd

# --- 1. 데이터 로드 ---
# 'fruits_300.npy' 파일에는 (300, 100, 100) 형태의 numpy 배열이 들어있습니다.
try:
    fruits = np.load('fruits_300.npy')
    print("성공적으로 'fruits_300.npy' 파일을 로드했습니다.")
    print(f"원본 데이터 형태: {fruits.shape}")
except FileNotFoundError:
    print("오류: 'fruits_300.npy' 파일을 찾을 수 없습니다. 코드와 같은 폴더에 파일을 위치시켜 주세요.")
    exit()

# --- 2. 데이터 형태 변환 ---
# TensorFlow Projector는 2차원 데이터를 입력으로 받습니다. (샘플 수, 특성 수)
# 100x100 이미지를 10000개의 픽셀 벡터로 길게 펼쳐줍니다.
# (300, 100, 100) -> (300, 10000)
fruits_2d = fruits.reshape(-1, 100 * 100)
print(f"변환된 데이터 형태: {fruits_2d.shape}")


# --- 3. 벡터(Vectors) 파일 저장 ---
# 변환된 이미지 픽셀 데이터를 tsv(Tab-Separated Values) 파일로 저장합니다.
# TensorFlow Projector는 tsv 파일을 기본으로 지원합니다.
vectors_filename = 'fruits_vectors.tsv'
pd.DataFrame(fruits_2d).to_csv(
    vectors_filename,
    sep='\t',      # 탭으로 열을 구분.
    header=False,  # 열 이름 저장 안함
    index=False    # 행 번호는 저장 안함
)
print(f"벡터 파일 '{vectors_filename}'이 성공적으로 저장되었습니다.")


# --- 4. 메타데이터(Metadata) 파일 저장 ---
# 각 이미지가 어떤 과일인지 라벨 정보를 만들어줍니다.
# 이 데이터셋은 사과 100개, 바나나 100개, 파인애플 100개 순서로 구성되어 있습니다.
labels = ['apple'] * 100 + ['banana'] * 100 + ['pineapple'] * 100
metadata_filename = 'fruits_metadata.tsv'
pd.DataFrame(labels, columns=['Label']).to_csv(
    metadata_filename,
    sep='\t',       # 탭으로 열을 구분
    index=False,    # 행 번호는 저장 안함
    header=False    # 열 이름 저장 안함
)
print(f"메타데이터 파일 '{metadata_filename}'이 성공적으로 저장되었습니다.")

**2. 사이트에 변화된 데이터 업로드하기**
- https://projector.tensorflow.org/ 사이트 접속
    
- PC에 다운로드된 파일 사이트에 업로드하기(왼쪽 uplad 버튼 클릭)
    - vector파일 : fruits_vectors.tsv
    - metadata파일 : fruits_metadata.tsv

**3. PCA 방법과 t-SNE 방법으로 데이터 확인하기**



---



## **2. t-SNE(t-distributed Stochastic Neighbor Embedding)**
분산 확률적 이웃 임베딩

- 고차원의 복잡한 데이터를 2차원 또는 3차원으로 축소하는 방법
- **비선형적인 방법의 차원 축소**(복잡한 곡선, 곡면 구조(Manifold)를 보존하면서 저차원으로 축소 -> Manifold Learning이라고도 불림)
- 복잡한 데이터의 시각화에 주로 사용 - 고차원 공간에서의 데이터들의 유사성과 저차원 공간에서의 데이터들의 유사성 계산

### **예제 : t-SNE 기본**

**1. 합성 데이터 생성하기**

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

# 실습용 데이터셋 생성하기
x, y = make_blobs(n_features=10,
                  n_samples=100,
                  centers=3,
                  random_state=42,
                  cluster_std=2)

print(x.shape)
print(y.shape)

plt.scatter(x[:, 0], x[:, 1])
plt.show()


**2. 2차원 t-SNE 시각화하기**

In [ ]:
import pandas as pd
from sklearn.manifold import TSNE

# 2차원 t-SNE 임베딩하기
tsne_np = TSNE(n_components=2,random_state=1).fit_transform(x)

# Numpy array를 DataFrame으로 변환하기
tsne_df = pd.DataFrame(tsne_np, columns = ['component 0', 'component 1'])
tsne_df

In [ ]:
import matplotlib.pyplot as plt

# class target 정보 불러오기
tsne_df['target'] = y

# target별 분리하기
tsne_df_0 = tsne_df[tsne_df['target'] == 0]
tsne_df_1 = tsne_df[tsne_df['target'] == 1]
tsne_df_2 = tsne_df[tsne_df['target'] == 2]

# target별 시각화하기
plt.scatter(tsne_df_0['component 0'], tsne_df_0['component 1'],
            color = 'pink', label = 'A')
plt.scatter(tsne_df_1['component 0'], tsne_df_1['component 1'],
            color = 'purple', label = 'B')
plt.scatter(tsne_df_2['component 0'], tsne_df_2['component 1'],
            color = 'yellow', label = 'C')

plt.xlabel('component 0')
plt.ylabel('component 1')
plt.legend()
plt.show()

**3. 3차원 t-SNE 시각화하기**

In [ ]:
# 3차원 t-SNE 임베딩하기
tsne_np = TSNE(n_components=3, random_state=15).fit_transform(x)

# Numpy array를 DataFrame으로 변환하기
tsne_df = pd.DataFrame(tsne_np,
        columns = ['component 0', 'component 1', 'component 2'])

print(tsne_df)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

# 3차원 그래프 세팅하기
fig = plt.figure(figsize=(9, 6))
ax = fig.add_subplot(111, projection='3d')

# class target 정보 불러오기
tsne_df['target'] = y

# target 별 분리하기
tsne_df_0 = tsne_df[tsne_df['target'] == 0]
tsne_df_1 = tsne_df[tsne_df['target'] == 1]
tsne_df_2 = tsne_df[tsne_df['target'] == 2]

# target 별 시각화하기
ax.scatter(tsne_df_0['component 0'],
           tsne_df_0['component 1'],
           tsne_df_0['component 2'],
           color = 'pink', label = 'A')
ax.scatter(tsne_df_1['component 0'],
           tsne_df_1['component 1'],
           tsne_df_1['component 2'],
           color = 'purple', label = 'B')
ax.scatter(tsne_df_2['component 0'],
           tsne_df_2['component 1'],
           tsne_df_2['component 2'],
           color = 'yellow', label = 'C')

ax.set_xlabel('component 0')
ax.set_ylabel('component 1')
ax.set_zlabel('component 2')
ax.legend()
plt.show()



---

